## Collecte de données

In [1]:
!pip3 install --user Pillow sparqlwrapper pandas requests

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os 
import json
from PIL import Image
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil

/Users/julieludwig/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


On créé un dossier image et on télécharge 100 images de notre choix.

In [ ]:
#os.mkdir("image")
endpoint_url = "https://query.wikidata.org/sparql"

query= """
SELECT DISTINCT ?horse ?horseLabel ?image WHERE {
    ?horse wdt:P31 wd:Q726.
     ?horse wdt:P18 ?image. 
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }
LIMIT 100
"""
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

array=[]
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
            (
                    result["horseLabel"]["value"],
                    result["image"]["value"],
            )
    )
dataframe = pd.DataFrame(array, columns=["car", "image"])
dataframe = dataframe.astype(
        dtype={"car": "<U200", "image": "<U200"}
)

def download_image(url, folder_path):
    headers = {"User-Agent": "mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        filename = os.path.basename(url)
        filepath = os.path.join(folder_path, filename)
        with open(filepath, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

folder_path = "./images"
dataframe.image.apply(lambda url: download_image(url, folder_path))

Création du tableau json contenant les métadonnées de chaque image du dossier images

In [6]:
import os 
import json
from PIL import Image

#récupération des données exif d'une image
def get_image_metadata(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            format = img.format
            exif_data = img._getexif()

            orientation = 'N/A'
            creation_date = 'N/A'
            camera_model = 'N/A'

            #si image contient des données exif: spécification des infos
            if exif_data:
                orientation = exif_data.get(0x0112) if 0x0112 in exif_data else 'N/A'
                creation_date = exif_data.get(0x0132, 'N/A') if 0x0132 in exif_data else 'N/A'
                camera_model = exif_data.get(0x0110, 'N/A') if 0x0110 in exif_data else 'N/A'
            
            metadata = {
                'filename': os.path.basename(image_path),
                'size': os.path.getsize(image_path),
                'format': format,
                'orientation': orientation,
                'width': width,
                'height': height,
                'creation_date': creation_date,
                'camera_model': camera_model
            } 
            return metadata
        
    except FileNotFoundError:
        print("Error: Image file not found: {image_path}")
        return {}
    
#parcourt les images du dossier image et crée un tableau avec les infos de ces images
dossier_images = "images"
metadata_images = []

for filename in os.listdir(dossier_images):
    image_path = os.path.join(dossier_images, filename)
    if os.path.isfile(image_path) and os.path.splitext(filename)[-1].lower() in ('.jpg', '.jpeg', '.png'):
        metadata = get_image_metadata(image_path)
        if metadata:
            metadata_images.append(metadata)

#tableau metadata_images dans le fichier json
json_path = "metadata_images.json"
with open(json_path, 'w') as json_file:
    json.dump(metadata_images, json_file, indent=4, default=str)
print("ok")

ok
